# 04 - Model Training

Train multiple ML/DL models for sales forecasting

In [ ]:
import pandas as pd
import numpy as np
import pickle
import sys
import os
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.ensemble import RandomForestRegressor, GradientBoostingRegressor
from sklearn.linear_model import LinearRegression

sys.path.append(os.path.abspath('../src'))
from src.train import train_model, save_model
from src.evaluate import calculate_metrics, print_metrics

## Load Featured Dataset

In [2]:
df = pd.read_csv('../data/processed/featured_dataset.csv')
print(f"Dataset shape: {df.shape}")
print(f"Columns: {list(df.columns)}")
print(f"\nFirst rows:")
df.head()

Dataset shape: (146608, 44)
Columns: ['date', 'item_id', 'quantity', 'price_base', 'sum_total', 'store_id', 'division', 'format', 'city', 'area', 'dept_name', 'class_name', 'subclass_name', 'item_type', 'weight_volume', 'weight_netto', 'fatness', 'price', 'code', 'promo_price_before', 'promo_price_after', 'promo_days', 'online_qty', 'online_price', 'online_revenue', 'markdown_normal_price', 'markdown_price', 'markdown_qty', 'year', 'month', 'quarter', 'day_of_week', 'day_of_month', 'week_of_year', 'quantity_lag_7', 'sum_total_lag_7', 'quantity_lag_30', 'sum_total_lag_30', 'quantity_rolling_mean_7', 'sum_total_rolling_mean_7', 'quantity_rolling_mean_14', 'sum_total_rolling_mean_14', 'quantity_rolling_mean_30', 'sum_total_rolling_mean_30']

First rows:


,date,item_id,quantity,price_base,sum_total,store_id,division,format,city,area,...,quantity_lag_7,sum_total_lag_7,quantity_lag_30,sum_total_lag_30,quantity_rolling_mean_7,sum_total_rolling_mean_7,quantity_rolling_mean_14,sum_total_rolling_mean_14,quantity_rolling_mean_30,sum_total_rolling_mean_30
0,2023-08-23,0be3804714de,8.000,129.90,1039.20,1,Div1,Regular,City1,1200,...,15.000,1948.50,4.000,389.70,7.142857,927.728571,10.785714,1390.882143,9.433333,1209.353000
1,2023-03-15,0be3804714de,3.000,129.90,389.70,1,Div1,Regular,City1,1200,...,15.000,1947.60,10.000,1280.61,5.428571,705.171429,9.285714,1196.771429,9.200000,1179.656000
2,2022-12-30,18d0dd039cd8,3.332,699.90,2332.08,1,Div1,Regular,City1,1200,...,1.516,1061.05,0.374,246.80,6.827429,3097.511429,5.230714,2523.572857,5.209267,2605.126333
3,2023-08-30,b55a901fb8ff,7.000,59.89,419.20,1,Div1,Regular,City1,1200,...,6.000,406.68,2.000,119.80,3.571429,223.922857,4.428571,276.397143,3.900000,239.049667
4,2024-08-22,0be3804714de,25.000,129.90,3247.50,1,Div1,Regular,City1,1200,...,6.000,779.40,3.000,389.70,8.142857,1057.757143,9.428571,1224.642857,9.933333,1274.916000


## Prepare Data

In [3]:
# Define target and features
target_col = 'quantity'  # Target variable for sales forecasting

# Separate features and target
X = df.drop(columns=[target_col])
y = df[target_col]

print(f"Features shape: {X.shape}")
print(f"Target shape: {y.shape}")
print(f"Target statistics:")
print(y.describe())

Features shape: (146608, 43)
Target shape: (146608,)
Target statistics:
count    146608.00000
mean          9.53005
std          42.07140
min         -28.00000
25%           2.00000
50%           3.25500
75%           8.00000
max        4243.00000
Name: quantity, dtype: float64


## Train-Test Split

In [4]:
# Split data
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

print(f"Train size: {X_train.shape[0]} samples")
print(f"Test size: {X_test.shape[0]} samples")
print(f"Training ratio: {X_train.shape[0] / len(df) * 100:.1f}%")
print(f"Test ratio: {X_test.shape[0] / len(df) * 100:.1f}%")

Train size: 117286 samples
Test size: 29322 samples
Training ratio: 80.0%
Test ratio: 20.0%


## Feature Scaling

In [5]:
scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_test_scaled = scaler.transform(X_test)

print(f"✓ Features scaled")
print(f"Train mean: {X_train_scaled.mean():.6f}")
print(f"Train std: {X_train_scaled.std():.6f}")

# Save scaler
with open('../models/scaler.pkl', 'wb') as f:
    pickle.dump(scaler, f)
print(f"✓ Scaler saved")

ValueError: could not convert string to float: '2024-01-28'

## Train Models

In [6]:
# Train multiple models
models = {}

print("Training models...\n")

# 1. Linear Regression
print("1. Linear Regression...")
lr_model = LinearRegression()
lr_model.fit(X_train_scaled, y_train)
models['linear_regression'] = lr_model
y_pred_lr = lr_model.predict(X_test_scaled)
print_metrics(calculate_metrics(y_test, y_pred_lr))

# 2. Random Forest
print("\n2. Random Forest Regressor...")
rf_model = RandomForestRegressor(n_estimators=100, random_state=42, n_jobs=-1)
rf_model.fit(X_train_scaled, y_train)
models['random_forest'] = rf_model
y_pred_rf = rf_model.predict(X_test_scaled)
print_metrics(calculate_metrics(y_test, y_pred_rf))

# 3. Gradient Boosting
print("\n3. Gradient Boosting Regressor...")
gb_model = GradientBoostingRegressor(n_estimators=100, random_state=42)
gb_model.fit(X_train_scaled, y_train)
models['gradient_boosting'] = gb_model
y_pred_gb = gb_model.predict(X_test_scaled)
print_metrics(calculate_metrics(y_test, y_pred_gb))

Training models...

1. Linear Regression...


NameError: name 'X_train_scaled' is not defined

## Model Comparison

In [7]:
from sklearn.metrics import mean_squared_error, mean_absolute_error, r2_score

results = {}
for name, model in models.items():
    y_pred = model.predict(X_test_scaled)
    results[name] = {
        'RMSE': np.sqrt(mean_squared_error(y_test, y_pred)),
        'MAE': mean_absolute_error(y_test, y_pred),
        'R2': r2_score(y_test, y_pred)
    }

results_df = pd.DataFrame(results).T
print("\nModel Comparison:")
print(results_df)

best_model_name = results_df['R2'].idxmax()
print(f"\n🏆 Best Model: {best_model_name}")
best_model = models[best_model_name]


Model Comparison:
Empty DataFrame
Columns: []
Index: []


KeyError: 'R2'

## Save Best Model

In [8]:
# Save the best model
model_path = '../models/best_model.pkl'
with open(model_path, 'wb') as f:
    pickle.dump(best_model, f)
print(f"✓ Best model saved: {model_path}")
print(f"  Model type: {type(best_model).__name__}")
print(f"  R² Score: {results_df.loc[best_model_name, 'R2']:.4f}")
print(f"  RMSE: {results_df.loc[best_model_name, 'RMSE']:.4f}")
print(f"  MAE: {results_df.loc[best_model_name, 'MAE']:.4f}")

NameError: name 'best_model' is not defined

## Feature Importance

In [9]:
if hasattr(best_model, 'feature_importances_'):
    importances = best_model.feature_importances_
    feature_importance_df = pd.DataFrame({
        'feature': X.columns,
        'importance': importances
    }).sort_values('importance', ascending=False)
    
    print("\nTop 10 Important Features:")
    print(feature_importance_df.head(10))
    
    # Plot
    import matplotlib.pyplot as plt
    fig, ax = plt.subplots(figsize=(10, 6))
    feature_importance_df.head(10).plot(x='feature', y='importance', kind='barh', ax=ax)
    plt.title('Top 10 Feature Importances')
    plt.tight_layout()
    plt.savefig('../reports/feature_importance.png', dpi=100)
    plt.show()
else:
    print("Selected model does not have feature importances")

NameError: name 'best_model' is not defined